<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
Principal Component Analysis
              
</p>
</div>

Data Science Cohort Live NYC July 2022
<p>Phase 4: Topic 33</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
    

In [ ]:
from scipy.special import gamma
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels as sm
from statsmodels.regression.linear_model import OLS
from sklearn.metrics import mean_squared_error as mse

%matplotlib inline

# Learning Goals

- Explain the concepts behind principal component analysis (PCA)
- Explain how PCA addresses the problem of multicollinearity
- Explain the idea of eigendecomposition
- Implement PCA using `sklearn`

#### Motivation: The curse of dimensionality

In [ ]:

%%capture cod

def sphere_calc(m, R):
    numerator = (R**m * np.pi**(m/2))
    denom = gamma(m/2 + 1)
    
    return numerator/denom

dim_list = pd.Series(np.arange(1,21))
sph_vol_list = dim_list.map(lambda M: sphere_calc(M, 2)) # get +- 2 std of standardized variables

density_vs_dim = 5000/sph_vol_list
fig, ax = plt.subplots()
sns.lineplot(x = density_vs_dim.index, y = density_vs_dim.values, ax = ax)
ax.set_ylabel('Point density')
ax.set_xlabel('Number of Features')
ax.set_title('Point density:  feature dimension scaling')
plt.show()


- Need good statistical sampling in feature space for training
- Dataset becomes sparse in high dimension. Hard to do statistical learning.


In [ ]:
cod()

Small numbers (low density):

- large sample-to-sample fluctuations.

High variance from dimensionality: hard to train model that generalizes.


Another issue in data with a ton of features:
- many of the features are likely correlated with each other
- high variance


## How could we address this problem?


Could keep all variables and regularize.
   

Or could figure out how to reduce dimension:

- construct smaller set of variables that are combinations of original variables
- largely account for variation scale and correlations of features with this smaller set
- i.e., reduce dim, preserve as much information as possible

#### Principal Component Analysis (PCA)

Principal Component Analysis (PCA) is a tool for reducing the dimensionality of our data in a way that tries to preserve information. It does this by projecting our data from a higher-dimensional space onto a lower-dimensional space. The PCA algorithm chooses a lower-dimensional space to project to that will preserve as much variance as possible from our original dataset.

<img src = "Images/PCA_basics.gif" width = 800 />

- Rotate to new coordinate system: 
    - directions characteristic of data spread in various dimensions.
    - new features uncorrelated (perpendicular)
- Find direction(s) of maximum variance:
    - project data onto this lower-D space.

Often doing this in much higher-D spaces.
- Very surprising how low the dimensionality capturing most of the feature variance can be.

Let's try an implement within given situation:

- Predict the cost to ship a package based on its features. 

In [ ]:
packages = pd.read_csv('data/packages.csv')
packages.head(3)

In [ ]:
packages.info()

- Quite a few features for a small dataset,
- Also: likely that physical dimensions are correlated

Check out those correlations:

In [ ]:
packages_scaled = (packages - packages.mean())/packages.std()

sns.heatmap(packages_scaled.corr(),
            annot=True,
            fmt='0.2g',
            vmin=-1,
            vmax=1,
            center=0,
            cmap='coolwarm');

Very high correlations between the different physical package characteristics.

- Some combination of variables as one variable
- Another set for the other

- One as strong mixture of physical features
- other dominated by Distance.
    

## Dimensionality
You can think about each variable as a dimension, and thus each package as a data point. If we take just one feature, we can easily visualize this in 2 dimensional space

In [ ]:
packages.plot(kind='scatter', y='Shipping Cost ($)', x='Length (in)');

You can think of each package as a point in six-dimensional space - 5 dimensions for our features and 1 for our target.

PCA: 
- relies on decomposing this **covariance matrix** $X^TX$ of scaled/centered data:

Notice that, for the centered data matrix $X$, the covariance matrix $C$ is equal to $\frac{1}{n-1}X^TX$:

In [ ]:
packages_scaled.cov()

In [ ]:
packages_scaled.T.dot(packages_scaled) / (len(packages_scaled)-1)

So first step of PCA

Rotate coordinates: constructs new features

Linear combination of old features:

$$ \begin{bmatrix} PC_1 \\ PC_2 \end{bmatrix} = \begin{bmatrix} \cos(\phi) & -\sin(\phi) \\ \sin(\phi) & \cos(\phi) \end{bmatrix}\begin{bmatrix} X_1 \\ X_2 \end{bmatrix}  $$

<img src = "Images/PCA_closeup.png" />

In rotated frame:
    
- New features not correlated.

<img src = "Images/pca_rot.jpg" width = 350 />

Getting variance matrix in new coordinate system:
- Get variance along specific PC1, PC2

Finds characteristic (eigen) directions of variance and gets (eigen) values of the variance along characteristic directions

## Eigendecomposition

We will use an **eigendecomposition** of the covariance matrix to create a new set of dimensions. We can then decide how many of these dimensions to keep based on how much variance is captured by each dimension.

Here, we show you how to do this using the NumPy `.eig()` function, but we will learn how to do PCA more easily in `sklearn` later.

In [ ]:
from numpy import linalg

y_packages_scaled = packages_scaled['Shipping Cost ($)']
X_packages_scaled = packages_scaled.drop('Shipping Cost ($)', axis=1)

cov_mat = X_packages_scaled.cov().values
eigvals, eigvecs = np.linalg.eig(cov_mat)

## Eigenvalues

**Eigenvalues** represent the relative amount of variance captured by each new dimension. The average eigenvalue will be 1, so we look for values over 1 to identify dimensions that capture more variance than average.

In [ ]:
eigvals

### Proportion of Variance


You can also divide your eigenvalues by the number of features and then interpret them as the _proportion of variance in the features_ captured by each dimension.

In [ ]:
eigvals/5

Covariance matrix in new, rotated coordinate system

In [ ]:
colname = ['PC' + str(n) for n in range(len(Xcov)) ]
Xcov_PCA = pd.DataFrame(np.diag(eigvals), index = colname, columns = colname)
Xcov_PCA

It looks like we have one great dimension capturing 3.4x more variance than average, one OK dimension capturing an average amount of variance, and three other dimensions that don't capture much variance. This is in line with what we were expecting! It means that we can just use the first two dimensions - and drop the last three - without losing much variance/information from our predictors.

## Eigenvectors (aka Principal Components)

**Eigenvectors** represent the new dimensions, which we call **principal components** when doing PCA. There is one eigenvector for each dimension, and they are all combined together into one matrix.

In [ ]:
eigvecs

In [ ]:
pc_comp = pd.DataFrame(eigvecs.T,
                       index = colname,
                       columns = X_train.columns)
pc_comp

In PCA, the values in our eigenvectors are called **component weights**, and they tell us how much variance of each feature is captured by that dimension. These weights range from -1 to 1, but the relative sizes are what matter.

PCs are mixtures of original features:
- In our case very equally distributed across different features.
- PCs are usually NOT interpretable

Looking at the variance and principal components side-by-side

In [ ]:
Xcov_PCA

In [ ]:
pc_comp

No question that PC0 and PC1 dominate variance:

- Most dominant always called **first principal component**
- Second most dominant: **second principal component**, etc.

- First principal component has high weight across all features.
- Second weights distance (not physical package attribute)

Together enough to capture most variance.

#### Orthogonality: formal way of saying something we know

The directions we found are **orthogonal** or perpendicular to each other:
- Same as saying: PCs are not correlated with each other.
- In such a case: dot products between different PC vectors vanish.

$$ \textbf{u}\cdot \textbf{v} = \textbf{u}^T\textbf{v} = \begin{bmatrix} u_1 & u_2 \end{bmatrix}\begin{bmatrix} v_1 \\ v_2 \end{bmatrix}  = 0 $$

$$ \begin{bmatrix} 1 & -1 \end{bmatrix}\begin{bmatrix} 1 \\ 1 \end{bmatrix}  = 0 $$


<img src = "Images/pca_rot.jpg" width = 350 />

What we did:

<center><img src = "Images/PCA_orth.png" width = 450 /></center>

In [ ]:
eigvec1 = eigvecs[:, 0]
eigvec2 = eigvecs[:, 1]
eigvec1.dot(eigvec2)

#### The final step: transforming the data

We will now use these principal components to create new features. 
- These features will be weighted sums (aka **linear combinations**) of existing features
- Using the component weights from the eigenvectors.

Take first three PCs only: these **dominate** variance:
- Capture most of feature space.
- Decorrelated variables. Variance not an issue.

We will now create feature using the first 3 PCS: 

<img src = "Images/PCA_basics.gif" width = 500 />

Projecting the data onto these principal components

### First Component

In [ ]:
eigvec1

Notice that the first four numbers are relatively large, while the fifth is near zero. This means that this first dimension is almost entirely capturing the shared variance in our four size features, as we hoped! It's also interesting to note that the weights for the four features are almost equal, so they are equally represented in this dimension.

Our first feature will be calculated as follows:

**PC1** = 0.492 * Length + 0.508 * Width + 0.508 * Height + 0.492 * Weight - 0.003 * Distance

We use a dot product between the data and the eigenvector to do the arithmetic for us.


In [ ]:
data_array = X_packages_scaled.values
pc1 = data_array.dot(eigvec1)
X_packages_pca = pd.DataFrame(data=pc1, columns=['PC1'])
X_packages_pca.head()

In [ ]:
pcs = data_array.dot(eigvecs)
X_packages_pca = pd.DataFrame(data=pcs, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
X_packages_pca.head(10)

- We have lessened collinearity issue.
- Also reduced dimensionality considerably.

In [ ]:
sns.heatmap(X_packages_pca.corr(),
            annot=True,
            fmt='0.2g',
            vmin=-1,
            vmax=1,
            center=0,
            cmap= 'coolwarm');

#### Modeling

Now let's compare linear regression models with...

* All five original features 
* All five new features
* Only 2 best new features
* Only 1 best new feature

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_packages_scaled, y_packages_scaled)

**Model 1: All base features**:

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
sm1 = OLS(exog=X_train, endog=y_train).fit()
sm1.rsquared

In [ ]:
y_test_hat = sm1.predict(X_test)
mse(y_test, y_test_hat)

**Model 2: All PCs**:

In [ ]:
tr_eigval, tr_eigvec = np.linalg.eig(X_train.cov())

X_tr_pca = X_train.dot(tr_eigvec)

sm2 = OLS(exog=X_tr_pca, endog=y_train).fit()
sm2.rsquared

In [ ]:
y_test_hat2 = sm2.predict(X_test.dot(tr_eigvec))
mse(y_test, y_test_hat2)

**Model 3 (Exercise!): Only PCs 1 and 2**:

<details>
    <summary>Answer</summary>
<code>sm3 = OLS(exog=X_tr_pca.loc[:, :2], endog=y_train).fit()
print(sm3.rsquared)
y_test_hat3 = sm3.predict(X_test.dot(tr_eigvec).loc[:, :2])
mse(y_test, y_test_hat3)</code>

## PCA in `sklearn`

As always, `sklearn` makes this all much easier, this time with the `PCA()` class.

In [ ]:
pca = PCA(n_components=2) # Check out how `n_components` works

X_packages_pca2 = pca.fit_transform(X_packages_scaled)

You can get the eigenvalues and eigenvectors out, too 

In [ ]:
pca.explained_variance_

In [ ]:
pca.explained_variance_ratio_

Sometimes the signs get flipped on the eigenvectors - don't worry about it. Think of "up" and "down" as both representing the same dimension, just in opposite directions.

In [ ]:
pca.components_

After PCA, you can use your transformed data as you would in any model

In [ ]:
lr_pca = LinearRegression()
lr_pca.fit(X_packages_pca2, y_packages_scaled)
lr_pca.score(X_packages_pca2, y_packages_scaled)

# Scenario: Car Properties

Use PCA to reduce the dimensionality of features in the example below: Predict car mpg using car properties. We've done the data prep. Now you practice the modeling, including scoring on the test set.

## Data Prep

In [ ]:
cars = pd.read_csv('data/cars.csv')

In [ ]:
cars.head()

In [ ]:
cars.dtypes

In [ ]:
cars[' cubicinches'].replace(' ', np.nan, inplace=True)
cars[' cubicinches'] = cars[' cubicinches'].astype(float)

In [ ]:
cars[' weightlbs'].replace(' ', np.nan, inplace=True)
cars[' weightlbs'] = cars[' weightlbs'].astype(float)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cars.drop('mpg', axis=1),
                                                    cars['mpg'],
                                                   random_state=20)

In [ ]:
ct1 = ColumnTransformer(transformers=[
    ('imputer', SimpleImputer(), [1, 3])],
    remainder='passthrough')

In [ ]:
ct2 = ColumnTransformer(transformers=[
    ('scaler', StandardScaler(), [0, 1, 2, 3, 4, 5]),
    ('ohe', OneHotEncoder(), [6])],
    remainder='passthrough')

In [ ]:
pipe = Pipeline(steps=[
    ('ct1', ct1),
    ('ct2', ct2)
])

In [ ]:
pipe.fit(X_train)

In [ ]:
X_tr_pp = pipe.transform(X_train)
X_te_pp = pipe.transform(X_test)

## First Model w/o PCA


In [ ]:
## Let's start with a linear regression

lr = LinearRegression().fit(X_tr_pp, y_train)

## Score on train

lr.score(X_tr_pp, y_train)

In [ ]:
## Score on test

lr.score(X_te_pp, y_test)

In [ ]:
# Get the coefficients of the best-fit hyperplane

lr.coef_

Thus, our best-fit hyperplane is given by:

$2.177\times in^3\_sd - 4.645\times lbs.\_sd - 1.555\times cyl\_sd - 1.154\times hp\_sd -  0.267\times time_{60}\_sd + 2.604\times yr\_sd + 0.708\times brand_{Europe} + 0.912\times brand_{Japan} - 1.620\times brand_{US}$

In [ ]:
cars_pca = PCA(n_components=3) 

X_train_new = cars_pca.fit_transform(X_tr_pp)
X_test_new = cars_pca.transform(X_te_pp)

In [ ]:
cars_pca.components_

The results of our PCA are as follows:

**PC1** = 0.465 * cubicinches_sd + 0.435 * weightlbs_sd + 0.449 * cylinders_sd + 0.454 * hp_sd - 0.349 * time-to-60_sd - 0.187 * year_sd - 0.068 * Europe - 0.073 * Japan + 0.140 * US

**PC2** = -0.099 * cubicinches_sd - 0.196 * weightlbs_sd - 0.131 * cylinders_sd + 0.006 * hp_sd - 0.125 * time-to-60_sd - 0.937 * year_sd + 0.129 * Europe + 0.022 * Japan - 0.152 * US

**PC3** = 0.141 * cubicinches_sd + 0.342 * weightlbs_sd + 0.187 * cylinders_sd - 0.144 * hp_sd + 0.851 * time-to-60_sd - 0.239 * year_sd + 0.043 * Europe - 0.132 * Japan + 0.089 * US

### Modeling with New Dimensions

Now that we have optimized our features, we can build a new model with them!

In [ ]:
lr_pca = LinearRegression()
lr_pca.fit(X_train_new, y_train)
lr_pca.score(X_train_new, y_train)

In [ ]:
X_test_new = cars_pca.transform(X_te_pp)

In [ ]:
lr_pca.score(X_test_new, y_test)

In [ ]:
lr_pca.coef_

Thus, our best-fit hyperplane is given by:

$-2.967\times PC1 - 1.162\times PC2 -2.486\times PC3$

Of course, since the principal components are just linear combinations of our original predictors, we could re-express this hyperplane in terms of those original predictors!

And if the PCA was worth anything, we should expect the new linear model to be *different from* the first!

Recall that we had:

**PC1** = 0.465 * cubicinches_sd + 0.435 * weightlbs_sd + 0.449 * cylinders_sd + 0.454 * hp_sd - 0.349 * time-to-60_sd - 0.187 * year_sd - 0.068 * Europe - 0.073 * Japan + 0.140 * US

**PC2** = -0.099 * cubicinches_sd - 0.196 * weightlbs_sd - 0.131 * cylinders_sd + 0.006 * hp_sd - 0.125 * time-to-60_sd - 0.937 * year_sd + 0.129 * Europe + 0.022 * Japan - 0.152 * US

**PC3** = 0.141 * cubicinches_sd + 0.342 * weightlbs_sd + 0.187 * cylinders_sd - 0.144 * hp_sd + 0.851 * time-to-60_sd - 0.239 * year_sd + 0.043 * Europe - 0.132 * Japan + 0.089 * US

Therefore, our new PCA-made hyperplane can be expressed as:

$-2.967\times(0.465\times in^3\_sd + 0.435\times lbs.\_sd + 0.449\times cyl\_sd + 0.454\times hp\_sd - 0.349\times time_{60}\_sd - 0.187\times yr\_sd - 0.068\times brand_{Europe} - 0.073\times brand_{Japan} + 0.140\times brand_{US})$ <br/> $- 1.162\times(-0.099\times in^3\_sd - 0.196\times lbs.\_sd - 0.131\times cyl\_sd + 0.006\times hp\_sd - 0.125\times time_{60}\_sd - 0.937\times yr\_sd + 0.129\times brand_{Europe} + 0.022\times brand_{Japan} - 0.152\times brand_{US})$ <br/> $- 2.486\times(0.141\times in^3\_sd + 0.342\times lbs.\_sd + 0.187\times cyl\_sd -0.144\times hp\_sd + 0.851\times time_{60}\_sd - 0.239\times yr\_sd + 0.043\times brand_{Europe} - 0.132\times brand_{Japan} + 0.089\times brand_{US})$

Let's make these calculations:

In [ ]:
print(f'cubicinches_sd coef: {round(lr_pca.coef_ @ cars_pca.components_[:, 0], 3)}')
print(f'weightlbs_sd coef: {round(lr_pca.coef_ @ cars_pca.components_[:, 1], 3)}')
print(f'cylinders_sd coef: {round(lr_pca.coef_ @ cars_pca.components_[:, 2], 3)}')
print(f'horsepower_sd coef: {round(lr_pca.coef_ @ cars_pca.components_[:, 3], 3)}')
print(f'timeto60_sd coef: {round(lr_pca.coef_ @ cars_pca.components_[:, 4], 3)}')
print(f'year_sd coef: {round(lr_pca.coef_ @ cars_pca.components_[:, 5], 3)}')
print(f'Europe coef: {round(lr_pca.coef_ @ cars_pca.components_[:, 6], 3)}')
print(f'Japan coef: {round(lr_pca.coef_ @ cars_pca.components_[:, 7], 3)}')
print(f'US coef: {round(lr_pca.coef_ @ cars_pca.components_[:, 8], 3)}')

So our best-fit hyperplane using PCA is:

$-1.616\times in^3\_sd - 1.913\times lbs.\_sd - 1.646\times cyl\_sd - 0.996\times hp\_sd - 0.933\times time_{60}\_sd + 2.237\times yr\_sd - 0.055\times brand_{Europe} + 0.517\times brand_{Japan} - 0.462\times brand_{US}$

Recall that our first linear regression model had:

$2.177\times in^3\_sd - 4.645\times lbs.\_sd - 1.555\times cyl\_sd - 1.154\times hp\_sd -  0.267\times time_{60}\_sd + 2.604\times yr\_sd + 0.708\times brand_{Europe} + 0.912\times brand_{Japan} - 1.620\times brand_{US}$

which is clearly a different hyperplane.